In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')


if os.path.exists("/content/drive/MyDrive/ai_kepek") == False:
    !mkdir /content/drive/MyDrive/ai_kepek

#@markdown A neved az ugyanaz mint a képeknél amit feltöltöttél

# ahogy elnevezted a képeket, a zárojel előtt
neved='misrori' #@param{type: 'string'}

#@markdown A model neve az amit a hugging facen van a profilodban
# ami a hugingfacen van usernev/modelnev
modeled_neve = 'misrori/misroriai' #@param{type: 'string'}

!nvidia-smi
!pip install diffusers
!pip install transformers scipy ftfy accelerate

import random
import string


from PIL import Image
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


from diffusers import DiffusionPipeline
import torch
pipeline = DiffusionPipeline.from_pretrained( modeled_neve, torch_dtype=torch.float16)

pipeline.to("cuda")



import pandas as pd
import random

df = pd.read_csv('https://raw.githubusercontent.com/misrori/ai/main/styles.tsv', sep='\t')
df = df.loc[df['style'].str.contains('Portrait', case=False)].reset_index(drop=True)


df['promt'] = [f"Square photo of {neved} person style of {x}, inspired by {x}" for x in df['nev']]
df['artist'] = df['nev'].str.replace(', ', '_').str.lower().str.replace(' ', '').str.replace('(', '_').str.replace(')', '_').str.replace('/', '_')

kep_id = 1
for index, row in df.iterrows():

    imgs = pipeline(row['promt'], height = 560, width=560, num_images_per_prompt = 2 ).images

    for img in imgs:
      img.save(f"/content/drive/MyDrive/ai_kepek/{row['artist']}_{kep_id}.png")
      kep_id+=1


os.system(f"ffmpeg -framerate 10 -pattern_type glob -i '/content/drive/MyDrive/ai_kepek/*.png' -c:v libx264 -pix_fmt yuv420p /content/drive/MyDrive/ai_kepek/video.mp4")
